In [ ]:
import os
import glob
import numpy as np
import pandas as pd

In [ ]:
path0 = r"..\Result" + "\\"
path_discharge = path0 + "Discharge" + "\\"
path_era = path0 + "ERA5_Land" + "\\"
path_temperture = r"..\01 Merge Data\Data" + "\\"

file_station1 = r"..\01 Merge Data\Table\Station Info2.csv"
file_station2 = r"Table\Station Info.csv"

path_data = r"Data" + "\\"

In [ ]:
df_station = pd.read_csv(file_station1)

# file_list_era = glob.glob(os.path.join(path_era, "*.csv"))
# file_list_era.sort()

In [ ]:
var_list1 = [
    "dewpoint_temperature_2m",
    "temperature_2m",
    "soil_temperature_level_1",
    "snow_depth_water_equivalent",
    "snowfall_sum",
    "snowmelt_sum",
    "sub_surface_runoff_sum",
    "surface_runoff_sum",
    "surface_net_solar_radiation_sum",
    "surface_net_thermal_radiation_sum",
    "total_evaporation_sum",
    "total_precipitation_sum",
    "u_component_of_wind_10m",
    "v_component_of_wind_10m",
]

var_list2 = [
    "Dewpoint_temperature_2m",
    "Temperature_2m",
    "Soil_temperature_1",
    "Snow_depth_water_equivalent",
    "Snowfall",
    "Snowmelt",
    "Sub_surface_runoff",
    "Surface_runoff",
    "Surface_net_solar_radiation",
    "Surface_net_thermal_radiation",
    "Evaporation",
    "Precipitation",
    "Wind_U",
    "Wind_V",
]

scale_var_list = [
    "Snow_depth_water_equivalent",
    "Snowfall",
    "Snowmelt",
    "Sub_surface_runoff",
    "Surface_runoff",
    "Surface_net_solar_radiation",
    "Surface_net_thermal_radiation",
    "Evaporation",
    "Precipitation",
]

prefix_list = ["p1"]

dem_list = ["Elevation", "Slope", "Aspect"]
dem_list2 = [f"{p}_{d}" for p in prefix_list for d in dem_list]

In [ ]:
window_sizes = [2, 15, 30, 45]  # Length of the moving window in days

date1 = pd.Timestamp(1950, 1, 1)
date2 = pd.Timestamp(2023, 12, 31)
time_index = pd.date_range(start=date1, end=date2)

date_missing = pd.Timestamp(1950, 1, 1)

In [ ]:
def preprocess_era5(df, prefix):
    for var in scale_var_list:
        df[f"{prefix}_{var}"] = df[f"{prefix}_{var}"] * 1e5

    df[f"{prefix}_Wind"] = np.sqrt(
        df[f"{prefix}_Wind_U"] ** 2 + df[f"{prefix}_Wind_V"] ** 2
    )
    df.drop(columns=[f"{prefix}_Wind_U", f"{prefix}_Wind_V"], inplace=True)

    # fill the missing values for Dec 31 with backward fill
    d1 = df.index.min()
    d2 = df.index.max()
    time_index0 = pd.date_range(start=d1, end=d2, freq="D")
    df = df.reindex(time_index0, method="ffill")

    return df


def encode_date_to_sin_cos(dates):
    if not isinstance(dates, pd.Series):
        raise ValueError("Input must be a pandas Series")

    dates = pd.to_datetime(dates)

    is_leap_year = dates.dt.is_leap_year
    day_of_year = dates.dt.dayofyear
    days_in_year = np.where(is_leap_year, 366, 365)

    JD_sin = np.sin(day_of_year * 2 * np.pi / days_in_year)
    JD_cos = np.cos(day_of_year * 2 * np.pi / days_in_year)

    return pd.Series(JD_cos)

In [ ]:
file_list = glob.glob(os.path.join(path_discharge, "*.csv"))
file_list.sort()
for file in file_list[:]:
    print(file)
    river = os.path.basename(file).split("__")[0]
    station = os.path.basename(file).split("__")[1][:-4]
    river2 = river.replace(" ", "_")
    station2 = (
        station.replace(" ", "_").replace(",", "").replace(".", "").replace("'", "")
    )
    river_station2 = f"{river2}__{station2}"

    row = df_station[
        (df_station["River"] == river) & (df_station["Station"] == station)
    ]
    assert len(row) == 1, print(river, station)
    catchment = row["Catchment"].values[0]
    width = row["Width"].values[0]

    # Read discharge data
    df_discharge = pd.read_csv(
        path_discharge + file,
        parse_dates=["Date"],
        index_col="Date",
    )
    try:
        df_discharge.rename(columns={"Discharge_Station": "Discharge"}, inplace=True)
    except:
        pass
    df_discharge = df_discharge[["Discharge"]]

    # Read DEM data
    df_dem = pd.read_csv(path_era + river_station2 + "__Point_DEM.csv")
    aspect_point = df_dem["p1_aspect"].values[0]
    elevation_point = df_dem["p1_elevation"].values[0]
    slope_point = df_dem["p1_slope"].values[0]

    # Read ERA5-Land data
    df_era = pd.read_csv(
        path_era + river_station2 + "__Point_ERA5_Land.csv",
        parse_dates=["date"],
        index_col="date",
    )
    df_era.index.name = "Date"
    df_era.drop(columns=["system:index", ".geo"], inplace=True)

    if date1 <= date_missing <= date2:
        # fill the values of "1950-01-01" with "1950-01-02"
        df_era.loc[date_missing] = np.nan
        df_era.sort_index(inplace=True)
        df_era.fillna(method="bfill", inplace=True)

    for prefix in prefix_list:
        col_list1 = [f"{prefix}_{col}" for col in var_list1]
        col_list2 = [f"{prefix}_{col}" for col in var_list2]
        df_era.rename(columns=dict(zip(col_list1, col_list2)), inplace=True)
        df_era = preprocess_era5(df_era, prefix)

    enviro_vars = df_era.columns.tolist() + ["Discharge"]

    df_era["p1_Elevation"] = elevation_point
    df_era["p1_Slope"] = slope_point
    df_era["p1_Aspect"] = aspect_point

    df_era["JD"] = df_era.index.dayofyear
    df_era["JD_cos"] = encode_date_to_sin_cos(df_era.index.to_series())

    cols = df_era.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df_era = df_era[cols]

    # Read Temperature data
    df_temperture = pd.read_csv(
        path_temperture + file, parse_dates=["Date"], index_col="Date"
    )
    df_temperture = df_temperture[["Temperature"]]

    # Merge discharge and ERA5-Land data
    df = pd.concat([df_era, df_discharge, df_temperture], axis=1, join="outer")
    df = df.resample("D").mean()

    df.insert(0, "River", river)
    df.insert(1, "Station", station)
    df["Catchment"] = catchment
    df["Width"] = width

    # Rolling mean
    var_list3 = df.columns.drop(
        ["River", "Station", "Temperature", "JD", "JD_cos", "Catchment", "Width"]
        + dem_list2
    ).tolist()

    for enviro_var in enviro_vars:
        for ws in window_sizes:
            df[f"{enviro_var}__{ws}"] = (
                df[enviro_var].rolling(window=ws, min_periods=1).mean()
            )

    df = df.copy()
    df = df.reindex(time_index)
    df.index.name = "Date"

    # Drop not rolling mean columns
    df = df.drop(columns=var_list3)
    df.to_csv(path_data + file)

In [ ]:
df